# Coffee Machine Power Roll Up

Link to the example in the Cookbook:\
[https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2_[...]5/views/_19_0_2_f46036d_1582648674041_192673_2975](https://mms.openmbee.org/alfresco/mmsapp/mms.html#/projects/PROJECT-ID_2_2_17_7_33_25_AM_3ccfaf88_159fe0d7ba9__7d4e_cae_tw_jpl_nasa_gov_127_0_0_1/master/documents/_18_5_2_e64033a_1513278979739_556269_28755/views/_19_0_2_f46036d_1582648674041_192673_2975)

## Structure
![image.png](https://mms.openmbee.org/alfresco/service/api/node/content/workspace/SpacesStore/483dd349-6ac8-4245-aa9a-e690815220b5/3fede08f-d5a1-4a08-b744-3f0d7ff77bd91583425515726..svg?alf_ticket=TICKET_e4c107ee1a7736e1c27056ef2968408747851114)
## Power Analysis
![image.png](https://mms.openmbee.org/alfresco/service/api/node/content/workspace/SpacesStore/df4736bf-4fcf-4c40-ac76-a4fe8a3eb62e/6cbdc315-2e46-49b9-81a1-2a461489b5121583425520646..svg?alf_ticket=TICKET_e4c107ee1a7736e1c27056ef2968408747851114)
## Power Analysis Context
![image.png](https://mms.openmbee.org/alfresco/service/api/node/content/workspace/SpacesStore/e9fbbde5-8049-4560-9eb1-476b8e091a1f/7a2df802-a693-4d8f-958d-6c35df9970c71582668970927..svg?alf_ticket=TICKET_e4c107ee1a7736e1c27056ef2968408747851114)

In [16]:
package 'CoffeeMaker Dynamic Power Roll-Up'{
    import SI::Real;
    import ScalarFunctions::sum;
    
    value type power;
    
    package 'Structure' {
        block 'Coffee Machine'  specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern'{
            part 'coffee Grinder' : 'Coffee Grinder' subsets subPower {
                perform 'Make Coffee'::grindBeans;
            }
            part 'coffee Pot' : 'Coffee Pot' subsets subPower {
                perform 'Make Coffee'::heatWater;
            }
            part brewer : 'Brewer' subsets subPower {
                perform 'Make Coffee'::brewCoffee;
            }
            part 'water Heater' : 'Water Heater' subsets subPower {
                perform 'Make Coffee'::keepWarm;
            }
            
            
            activity 'Make Coffee'(){
                first start;
                then action grindBeans : GrindBeans;
                
                then action heatWater : HeatWater;
                
                then action brewCoffee : BrewCoffee {
                    first start;
                    then action filterCoffee : FilterCoffee;
                    then done;
                }
                
                then action keepWarm : KeepWarm;
                then done;
            }
            
            
            activity PowerDraw{
                value power_draw : power = 0;
            }
            
            activity GrindBeans specializes PowerDraw {
                value redefines power_draw = 5;
            }
            activity HeatWater specializes PowerDraw {
                value redefines power_draw = 10;
            }
            activity BrewCoffee specializes PowerDraw {
                value redefines power_draw = 2;
            }
            activity FilterCoffee specializes PowerDraw {
                value redefines power_draw = 1;
            }
            activity KeepWarm specializes PowerDraw {
                value redefines power_draw = 1;
            }

        }
        
        block 'Coffee Grinder'  specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern';
        
        block 'Coffee Pot' specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern';
        
        block 'Brewer' specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern'{
            part filter : 'Filter' subsets subPower {
                perform 'Coffee Machine'::'Make Coffee'::brewCoffee::filterCoffee;
            }
        }
        
        block 'Water Heater' specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern';
        
        block 'Filter' specializes 'Power Roll-Up Pattern'::'PowerRollUpPattern';
    }
    
    package 'Power Roll-Up Pattern' {
        
        constraint def TotalChildren (child : Real[*], total : Real, offset : Real) {
            total == sum(child) + offset
        }
        
        constraint def power_per_time_calc (p1 : Real, p2 : Real) {
            p1 == p2 // * 1/SIMULATION_TIME
        }
        constraint def equals (p1 : Real, p2 : Real){
            p1 == p2
        }
        block 'Power Analysis Context'{
            value totalPower : Real;
            value power_per_time : Real;
            
            part 'coffee Machine' : Structure::'Coffee Machine';
            
            constraint power_calc : power_per_time_calc(
                p1 = power_per_time,
                p2 = 'coffee Machine'::totalPower
            );
            constraint assert_equal : equals(
                p1 = totalPower,
                p2 = 'coffee Machine'::totalPower
            );
        } //END block 'Power Analysis Context'
        
        block 'PowerRollUpPattern'{
            constraint sum : TotalChildren (
                child = subPower::totalPower, 
                total = totalPower,
                offset = power);
            
            part subPower : 'PowerRollUpPattern'[*];
            
            value 'totalPower' : power;
            value 'power' : Real[1];         
        } //END block 'Data Roll-up Aspect'
    }
}

Package CoffeeMaker Dynamic Power Roll-Up (1de06ba6-a77c-4c1d-aeb6-4d77eed4bc25)
